In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from collections import deque
import mplfinance 
from renkodf import Renko
from scipy.signal import lfilter
import fx # personal functions
# import MetaTrader5 as mt5

In [2]:
yr_tickstory_csvs = []
for year in range(16, 22):
    yr_tickstory_csvs.append(f"EURUSD_11_15_{year}_to_11_15_{year+1}.csv")
yr_tickstory_csvs

['EURUSD_11_15_16_to_11_15_17.csv',
 'EURUSD_11_15_17_to_11_15_18.csv',
 'EURUSD_11_15_18_to_11_15_19.csv',
 'EURUSD_11_15_19_to_11_15_20.csv',
 'EURUSD_11_15_20_to_11_15_21.csv',
 'EURUSD_11_15_21_to_11_15_22.csv']

In [5]:
# collect the actual data from csv file
# filename = "C:/Users/WilliamFetzner/Documents/Trading/DAT_XLSX_EURUSD_M1_2023.xlsx"

# exported from tickstory
filepath = "C:/Users/WilliamFetzner/Documents/Trading/"
yr_tickstory_csvs = []
for year in range(16, 22):
    yr_tickstory_csvs.append(f"EURUSD_11_15_{year}_to_11_15_{year+1}.csv")
tickstory_csvs = ["EURUSD_11_15_22_to_11_14_23.csv", "3mo_EURUSD.csv", "EURUSD_2_13_to_3_6.csv", 
                    "EURUSD_3_6_to_3_8_24.csv", "EURUSD_3_8_to_3_12_24.csv", "EURUSD_3_12_to_3_22_24.csv",
                    "EURUSD_3_23_to_3_30_24.csv"]
tickstory_csvs.extend(yr_tickstory_csvs)
tickstory_csvs


['EURUSD_11_15_22_to_11_14_23.csv',
 '3mo_EURUSD.csv',
 'EURUSD_2_13_to_3_6.csv',
 'EURUSD_3_6_to_3_8_24.csv',
 'EURUSD_3_8_to_3_12_24.csv',
 'EURUSD_3_12_to_3_22_24.csv',
 'EURUSD_3_23_to_3_30_24.csv',
 'EURUSD_11_15_16_to_11_15_17.csv',
 'EURUSD_11_15_17_to_11_15_18.csv',
 'EURUSD_11_15_18_to_11_15_19.csv',
 'EURUSD_11_15_19_to_11_15_20.csv',
 'EURUSD_11_15_20_to_11_15_21.csv',
 'EURUSD_11_15_21_to_11_15_22.csv']

In [ ]:
df = pd.DataFrame()
for df_name in tickstory_csvs:
    df_new = pd.read_csv(filepath + df_name)
    print(len(df_new))
    df = pd.concat([df, df_new], ignore_index=True)

In [ ]:
df = df.reset_index(drop=True)
# drop any duplicate rows
df = df.drop_duplicates()

In [ ]:
# convert date to datetime
df['datetime'] = pd.to_datetime(df['Timestamp'], format='%Y%m%d %H:%M:%S:%f')
# sort by datetime
df = df.sort_values(by='datetime')
# rename bid price to close
df.rename(columns={'Bid price':'close'}, inplace=True)
df_ready = df.set_index('datetime')
# adjust the datetime 7 hrs ahead to match market time
df_ready.index = df_ready.index + pd.Timedelta(hours=7)
ohlc = df_ready[['close']]
ohlc